In [34]:
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd

from preprocessing.generate_negatives import gen_neg_sample

In [35]:
test_edges = pd.read_parquet('data/test_edges.parquet')
test_features = pd.read_parquet('data/test_features.parquet')
test_labels = pd.read_parquet('data/test_labels.parquet')

val_edges = pd.read_parquet('data/val_edges.parquet')
val_features = pd.read_parquet('data/val_features.parquet')
val_labels = pd.read_parquet('data/val_labels.parquet')

train_edges = pd.read_parquet('data/train_edges.parquet')
train_features = pd.read_parquet('data/train_features.parquet')
train_labels = pd.read_parquet('data/train_labels.parquet')

In [36]:
train_labels_neg = gen_neg_sample(train_edges, train_labels)
val_labels_neg = gen_neg_sample(val_edges, val_labels)
test_labels_neg = gen_neg_sample(test_edges, test_labels)

test_edges.shape:  (8675721, 3)
test_labels.shape:  (330526, 3)
hist_purchases.shape:  (9006247, 2)  - Is total:  True
test_edges.shape:  (9006247, 3)
test_labels.shape:  (330839, 3)
hist_purchases.shape:  (9337086, 2)  - Is total:  True
test_edges.shape:  (9337086, 3)
test_labels.shape:  (328203, 3)
hist_purchases.shape:  (9665289, 2)  - Is total:  True


In [38]:
train_labels_neg

,user_id,product_id,label
0,1,46149,1
1,1,49235,1
2,1,32022,0
3,1,44593,0
4,2,47186,0
...,...,...,...
661047,206206,12902,0
661048,206209,48370,1
661049,206209,40310,1
661050,206209,27857,0


In [39]:
temp = pd.merge(train_labels_neg, val_labels_neg, on=["user_id", "product_id"])

In [40]:
temp

,user_id,product_id,label_x,label_y
0,2101,27966,0,1
1,4675,47081,0,0
2,5908,36984,0,0
3,15783,27061,0,0
4,18644,38007,0,0
5,18707,4913,0,1
6,23010,9076,0,1
7,24377,1702,0,0
8,24462,38866,0,0
9,25929,46651,0,1


In [41]:
temp['label_x'].value_counts()

0    60
Name: label_x, dtype: int64

In [42]:
val_labels_neg

,user_id,product_id,label
0,2,79,1
1,2,42231,0
2,2,16558,0
3,2,32242,0
4,2,31955,0
...,...,...,...
661673,206209,16168,1
661674,206209,14727,1
661675,206209,5622,1
661676,206209,23554,0


In [43]:
test_labels_neg

,user_id,product_id,label
0,1,35951,1
1,1,38928,1
2,1,39657,1
3,1,1838,0
4,1,30792,0
...,...,...,...
656401,206209,31477,1
656402,206209,22920,1
656403,206209,6567,1
656404,206209,28818,0


In [44]:
temp2 = pd.merge(val_labels_neg, test_labels_neg, on=["user_id", "product_id"])
temp2

,user_id,product_id,label_x,label_y
0,1313,33724,0,0
1,4165,43777,0,0
2,6118,26209,0,1
3,8062,44422,0,1
4,12081,26620,0,1
...,...,...,...,...
62,187856,47209,0,1
63,191833,30822,0,0
64,193943,4605,0,1
65,204229,19344,0,0


In [45]:
temp2['label_x'].value_counts()

0    67
Name: label_x, dtype: int64

In [11]:
train_labels_neg.to_parquet("data/train_labels_neg.parquet", index=None)
val_labels_neg.to_parquet("data/val_labels_neg.parquet", index=None)
test_labels_neg.to_parquet("data/test_labels_neg.parquet", index=None)

In [7]:
class BipartiteData(Data):
    def __init__(self, edge_index=None, x_s=None, x_t=None):
        super().__init__()
        self.edge_index = edge_index
        self.x_s = x_s
        self.x_t = x_t
    
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index':
            return torch.tensor([[self.x_s.size(0)], [self.x_t.size(0)]])
        else:
            return super().__inc__(key, value, *args, **kwargs)

In [12]:
edge_index = test_edges.loc[:, ["user_id", "product_id"]].to_numpy().T

In [13]:
data = BipartiteData(edge_index)